In [ ]:
import pandas as pd
import networkx as nx
import random

In [ ]:
# Import the preprocessed CSV files
train_df = pd.read_csv('/Users/csykianakis/Desktop/data_challenge/Data Challenge/preprocessed_train_data.csv')
test_df = pd.read_csv('/Users/csykianakis/Desktop/data_challenge/Data Challenge/preprocessed_test_data.csv')

X_all = train_df['text']
y_all = train_df['label']
X_test = test_df['text']

In [ ]:
#Read edgelist.txt
edgelist = pd.read_csv('../input/data-chall-train-test-edgelist/edgelist.txt', delimiter=' ',header=None,names=["source", "target"])

# Create a directed graph
G = nx.read_edgelist('../input/data-chall-train-test-edgelist/edgelist.txt', delimiter=' ', create_using=nx.DiGraph())

# print('Number of nodes:', G.number_of_nodes())
# print('Number of edges:', G.number_of_edges())

In [ ]:
for index, row in train_df.iterrows():
    G.add_node(row['domain'], text=row['text'])

# Step 2: Identify URLs without text
urls_without_text = [node for node in G.nodes() if not G.nodes[node].get('text')]

# Step 3: Find first-degree neighbors of URLs without text
neighbors_with_text = []
for url in urls_without_text:
    neighbors = list(G.neighbors(url))
    for neighbor in neighbors:
        if G.nodes[neighbor].get('text'):
            neighbors_with_text.append(neighbor)

# Step 4: Randomly select text from first-degree neighbors and truncate to a maximum of 1000 words
def truncate_text(text, max_words=512):
    words = text.split()
    if len(words) > max_words:
        return ' '.join(words[:max_words])
    return text

text_choices = [truncate_text(G.nodes[neighbor]['text']) for neighbor in neighbors_with_text]

# Step 5: Assign randomly selected truncated text to URLs without text
for url in urls_without_text:
    if text_choices:  # Check if there are any text choices available
        random_text = random.choice(text_choices)
        G.nodes[url]['text'] = random_text
        train_df.loc[train_df['domain'] == url, 'text'] = random_text


In [ ]:
train_df.to_csv('train_df_filled_null.csv', index=False)

In [ ]:
for index, row in test_df.iterrows():
    G.add_node(row['domain'], text=row['text'])

# Step 2: Identify URLs without text
urls_without_text = [node for node in G.nodes() if not G.nodes[node].get('text')]

# Step 3: Find first-degree neighbors of URLs without text
neighbors_with_text = []
for url in urls_without_text:
    neighbors = list(G.neighbors(url))
    for neighbor in neighbors:
        if G.nodes[neighbor].get('text'):
            neighbors_with_text.append(neighbor)

# Step 4: Randomly select text from first-degree neighbors and truncate to a maximum of 1000 words
def truncate_text(text, max_words=512):
    words = text.split()
    if len(words) > max_words:
        return ' '.join(words[:max_words])
    return text

text_choices = [truncate_text(G.nodes[neighbor]['text']) for neighbor in neighbors_with_text]

# Step 5: Assign randomly selected truncated text to URLs without text
for url in urls_without_text:
    if text_choices:  # Check if there are any text choices available
        random_text = random.choice(text_choices)
        G.nodes[url]['text'] = random_text
        test_df.loc[test_df['domain'] == url, 'text'] = random_text

In [ ]:
test_df.to_csv('test_df_filled_null.csv', index=False)